In [14]:
from rich.pretty import pprint

## The SGD Benchmark
Let's take a look at SGD. This is a benchmark for controlling the learning rate and momentum of networks for image classification. First, let's make in instance of the benchmark:

In [15]:
from dacbench.benchmarks import SGDBenchmark
bench = SGDBenchmark()

Now let's take a look at the elements of the config in this benchmark:

In [16]:
pprint(list(bench.config.keys()))

[
│   'config_space',
│   'observation_space_class',
│   'observation_space_type',
│   'observation_space_args',
│   'reward_range',
│   'device',
│   'use_instance_generator',
│   'cutoff',
│   'loss_function',
│   'loss_function_kwargs',
│   'use_momentum',
│   'seed',
│   'crash_penalty',
│   'multi_agent',
│   'instance_set_path',
│   'benchmark_info',
│   'epoch_mode',
│   'local_model_path',
│   'dataset_config'
]

The 'benchmark_info' tells us some things about this benchmark already:

In [17]:
pprint(bench.config["benchmark_info"])

{
│   'identifier': 'LR',
│   'name': 'Learning Rate Adaption for Neural Networks',
│   'reward': 'Negative Log Differential Validation Loss',
│   'state_description': ['Step', 'Loss', 'Validation Loss', 'Crashed'],
│   'action_description': ['Learning Rate', 'Momentum']
}

The 'config_space' specifies which actions are taken, i.e. which hyperparameters are configured. We can see that two floats, the "learning_rate" and the "momentum" are configured.

In [18]:
pprint(bench.config["config_space"])

Configuration space object:
  Hyperparameters:
│   learning_rate, Type: UniformFloat, Range: [0.0, 0.05], Default: 0.025
│   momentum, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5

The reward in this task has the following reward range:

In [19]:
pprint(bench.config["reward_range"])

[-1000000000, 1000000000]

Finally, the cutoff shows how many steps of the sequence are necessary for solution:

In [20]:
pprint(bench.config["cutoff"])

100.0

The config also contains some standard keys like the seed, instance set, multi agent mode or observation space config. The observation space usually does not need to be configured at all while the seed should be varied between runs. Multi agent mode has its own example which better shows how it works. There are also several benchmark-specific options:
- 'device': can be used to specify a device, e.g. a gpu, for this benchmark to run on.
- 'use_instance_generator': if True, instances won't be taken from the instance set but generated randomly on the fly.
- 'loss_function': loss function to use. The default is negative log likelihood.
- 'loss_function_kwargs': arguments for an alternative loss function.
- 'use_momentum': whether to use momentum.
- 'crash_penalty': penalty in case the optimization produces numerical errors.
- 'epoch_mode': if True, a step is a full epoch. If False, a step is a single backward pass.
- 'local_model_path': when using torchhub, this option can be used to specify a path to a downloaded version of the model.
- 'dataset_config': alternate dataset config dict. This can be used to add more datasets or different transformation options.

## SGD Instances
Now let's take a look at how a SGD instance looks. To do so, we first read the default instance set and look at its first element:

In [21]:
pprint(bench.config["instance_set_path"])
bench.read_instance_set()
pprint(bench.config.instance_set[0])

'sgd_cifar10_variations_train.csv'

SGDInstance(
│   model=<function create_model_from_layer_description.<locals>.make_model at 0x17f769240>,
│   optimizer_type='AdamW',
│   optimizer_params={},
│   dataset_path=PosixPath('/Users/theeimer/Documents/git/DACBench/dacbench/benchmarks'),
│   dataset_name='CIFAR10',
│   batch_size=1024,
│   fraction_of_dataset=1,
│   train_validation_ratio=0.8,
│   seed=0
)

As you can see, the instance contains a "model" constructor method, an "optimizer_type" and matching keyword arguments, a path and name to the classification dataset, the "batch_size" to use, the "fraction_of_dataset" to train on, the "train_validation_ratio" and the training seed.

## Running SGD
Lastly, let's look at the SGD benchmark in action. The state is a dictionary initialized at 0 for most values:

In [22]:
env = bench.get_environment()
pprint(env.reset())

Files already downloaded and verified


({'step': 0, 'loss': 0, 'validation_loss': 0, 'validation_accuracy': 0, 'done': False, 'average_loss': 0}, {})

If we take a step, we see the updated state:

In [23]:
action = env.action_space.sample()
state, reward, terminated, truncated, info = env.step(action)
pprint(state)

{
│   'step': 1,
│   'loss': 1.7135250568389893,
│   'validation_loss': array(1.7852292, dtype=float32),
│   'validation_accuracy': array(0.2939453, dtype=float32),
│   'done': False,
│   'average_loss': 2.0345866423606873
}

Furthermore, we also get a reward and termination and truncation signals. Truncation will be set to true after the number of steps exceeds the cutoff.

In [24]:
pprint(f"Reward {reward}")
pprint(f"Terminated {terminated}")
pprint(f"Truncated {truncated}")

'Reward -0.0'

'Terminated False'

'Truncated False'